In [247]:
import numpy as np
import pandas as pd
import utils
import pymer4.models
import ast
import matplotlib.pyplot as plt

In [115]:
exp_measures = pd.read_csv("data/measures.csv")

condition_map = utils.get_nudge_condition_map()
exp_measures["a_values"] = exp_measures.a_values.apply(ast.literal_eval).apply(tuple)
exp_measures["condition"] = pd.Categorical(exp_measures["a_values"].map(condition_map), categories=condition_map.values(), ordered=True)
# exp_measures["condition_coded"] = exp_measures["condition"].cat.codes-2
exp_measures["a_values"] = exp_measures["a_values"].apply(str)

In [368]:
column_names = {"Estimate": "$b$", "Z-stat": "$z$", "P-val": "$p$", "T-stat": "$t$", "F-stat": "$F$", "2.5_ci": "CI 2.5\%", "97.5_ci": "CI 97.5\%", "NumDF": "df"}
var_names = {"tta_0": "$\\textrm{TTA}_0$",
             "decision:tta_0": "decision:$\\textrm{TTA}_0$",
             "condition:tta_0": "condition:$\\textrm{TTA}_0$",
             "decision:condition:tta_0": "decision:condition:$\\textrm{TTA}_0$",
             "condition1": "Long acceleration",
             "condition2": "Acceleration nudge",
             "condition3": "Deceleration nudge",
             "condition4": "Long deceleration",
             "decision1": "decision"}

# Decision

In [248]:
model_is_go_decision = pymer4.models.Lmer("is_go_decision ~ tta_0 + condition + (1 | subj_id) ", data=exp_measures, family="binomial")
model_is_go_decision_fit = model_is_go_decision.fit(factors={"condition": ["Constant speed", "Long acceleration", "Acceleration nudge", "Deceleration nudge", "Long deceleration"]})
print(model_is_go_decision.coefs)

D:\source\cognitive-av\venv\lib\site-packages\rpy2\robjects\pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: is_go_decision~tta_0+condition+(1|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 3800	 Groups: {'subj_id': 19.0}

Log-likelihood: -1543.917 	 AIC: 3101.833

Random effects:

                Name    Var    Std
subj_id  (Intercept)  4.321  2.079

No random effect correlations specified

Fixed effects:

             Estimate     2.5_ci   97.5_ci        SE        OR  OR_2.5_ci  \
(Intercept) -9.370903 -10.758697 -7.983109  0.708071  0.000085   0.000021   
tta_0        1.825554   1.625594  2.025513  0.102022  6.206230   5.081438   
condition1  -0.767809  -1.052470 -0.483147  0.145238  0.464029   0.349074   
condition2  -0.113153  -0.393691  0.167386  0.143134  0.893014   0.674562   
condition3   0.504310   0.222531  0.786089  0.143767  1.655842   1.249234   
condition4   1.477320   1.183432  1.771207  0.149945  4.381186   3.265564   

             OR_97.5_ci      Prob  Prob_2.5_ci  Prob_97.5_ci     Z-stat  \
(Intercept)    0.000341  0.000085     0.000021

In [193]:
marginal_estimates, comparisons = model_is_go_decision.post_hoc(marginal_vars=["condition"])

P-values adjusted by tukey method for family of 10 estimates


In [194]:
marginal_estimates

,condition,Estimate,2.5_ci,97.5_ci,SE,DF
0,Constant speed,-0.243,-1.200,0.714,0.488,inf
1,Long acceleration,-1.011,-1.969,-0.053,0.489,inf
2,Acceleration nudge,-0.356,-1.313,0.600,0.488,inf
3,Deceleration nudge,0.261,-0.696,1.218,0.488,inf
4,Long deceleration,1.234,0.275,2.193,0.489,inf


In [195]:
comparisons

,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,Z-stat,P-val,Sig
0,Constant speed - Long acceleration,0.768,0.372,1.164,0.145,inf,5.287,0.000,***
1,Constant speed - Acceleration nudge,0.113,-0.277,0.504,0.143,inf,0.791,0.933,
2,Constant speed - Deceleration nudge,-0.504,-0.896,-0.112,0.144,inf,-3.508,0.004,**
3,Constant speed - Long deceleration,-1.477,-1.886,-1.068,0.150,inf,-9.852,0.000,***
4,Long acceleration - Acceleration nudge,-0.655,-1.050,-0.259,0.145,inf,-4.517,0.000,***
5,Long acceleration - Deceleration nudge,-1.272,-1.674,-0.870,0.147,inf,-8.626,0.000,***
6,Long acceleration - Long deceleration,-2.245,-2.671,-1.819,0.156,inf,-14.388,0.000,***
7,Acceleration nudge - Deceleration nudge,-0.617,-1.010,-0.224,0.144,inf,-4.286,0.000,***
8,Acceleration nudge - Long deceleration,-1.590,-2.001,-1.180,0.151,inf,-10.557,0.000,***
9,Deceleration nudge - Long deceleration,-0.973,-1.376,-0.570,0.148,inf,-6.591,0.000,***


In [212]:
df = model_is_go_decision.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]

p_func = lambda p: "{:.2f}".format(p) if p>0.001 else "$<0.001$"

df["P-val"] = df["P-val"].apply(p_func)

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('figures/tab_decision.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision", caption="Coefficients of the mixed-effects logistic regression describing the final decision as a function of $\\textrm{TTA}_0$ and condition (with Constant speed as the reference category). Participant ID was included as a random intercept."
     )
)

# Response time

Full model with three-way interactions

In [305]:
# model_RT = pymer4.models.Lmer("RT ~ 1 + decision * tta_0 * condition + (1 | subj_id)", data=exp_measures, family="gaussian")
model_RT = pymer4.models.Lmer("RT ~ 1 + decision + tta_0 + condition + decision:tta_0 + decision:condition + condition:tta_0 + decision:condition:tta_0 + (1 | subj_id)", data=exp_measures, family="gaussian")
model_RT_fit = model_RT.fit(factors={"condition": ["Constant speed", "Long acceleration", "Acceleration nudge", "Deceleration nudge", "Long deceleration"], "decision": ["Stay", "Go"]}, summarize=False)
# df = model_RT.coefs.loc[:, ["Estimate", "2.5_ci", "97.5_ci", "T-stat", "P-val", "Sig"]]

D:\source\cognitive-av\venv\lib\site-packages\rpy2\robjects\pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


**NOTE**: Column for 'residuals' not created in model.data, but saved in model.resid only. This is because you have rows with NaNs in your data.

**NOTE** Column for 'fits' not created in model.data, but saved in model.fits only. This is because you have rows with NaNs in your data.



In [306]:
RT_anova = model_RT.anova()
RT_anova.index = ["decision", "tta_0", "condition", "decision:tta_0", "decision:condition", "condition:tta_0", "decision:condition:tta_0"]

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Using original model contrasts, orthogonality not guaranteed)


In [290]:
RT_anova

,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
decision,6.699902,6.699902,1,2978.930242,20.093258,7.653882e-06,***
tta_0,10.986597,10.986597,1,2982.707584,32.949217,1.041041e-08,***
condition,2.999869,0.749967,4,2973.874348,2.249180,6.144640e-02,.
decision:tta_0,0.000071,0.000071,1,2977.777514,0.000212,9.883909e-01,
decision:condition,9.882377,2.470594,4,2974.004193,7.409405,6.184003e-06,***
condition:tta_0,4.669410,1.167353,4,2973.998393,3.500934,7.373453e-03,**
decision:tta_0:condition,9.943541,2.485885,4,2974.159973,7.455263,5.679566e-06,***


In [304]:
df = RT_anova.loc[:, ["SS", "MS", "NumDF", "F-stat", "P-val"]]

p_func = lambda p: "{:.2f}".format(p) if p>0.001 else "$<0.001$"
df["P-val"] = df["P-val"].apply(p_func)
#
# df["order"] = range(len(df))
# df.loc["\\textit{Go decisions}"] = ["", "", "", "", "", 1.5]
# df.loc["\\textit{Stay decisions}"] = ["", "", "", "", "", 6.5]
# df = df.sort_values(["order"]).drop(columns=["order"])

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('figures/tab_RT.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:RT", caption="ANOVA table based on the mixed-effects linear regression describing response time as a function of decision, $\\textrm{TTA}_0$, and condition."
     )
)

# Post hocs

In [319]:
marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=["decision"])
# "Cell" means of the ANOVA
marginal_estimates

,decision,Estimate,2.5_ci,97.5_ci,SE,DF
0,Stay,1.639,1.498,1.779,0.067,19.405
1,Go,0.577,0.435,0.718,0.068,20.107


In [197]:
comparisons

,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
0,Stay - Go,1.068,1.013,1.124,0.028,2963.701,37.832,0.0,***


In [344]:
marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=["condition"], grouping_vars=["decision"])
# "Cell" means of the ANOVA
marginal_estimates

P-values adjusted by tukey method for family of 10 estimates


,condition,decision,Estimate,2.5_ci,97.5_ci,SE,DF
0,Constant speed,Stay,1.634,1.485,1.783,0.072,25.839
1,Long acceleration,Stay,1.559,1.411,1.707,0.072,25.327
2,Acceleration nudge,Stay,1.509,1.360,1.658,0.072,25.886
3,Deceleration nudge,Stay,1.696,1.546,1.847,0.073,27.418
4,Long deceleration,Stay,1.794,1.633,1.955,0.080,38.005
5,Constant speed,Go,0.528,0.372,0.683,0.076,32.073
6,Long acceleration,Go,0.484,0.320,0.648,0.081,40.762
7,Acceleration nudge,Go,0.562,0.405,0.720,0.077,33.795
8,Deceleration nudge,Go,0.523,0.371,0.676,0.074,28.968
9,Long deceleration,Go,0.787,0.637,0.936,0.073,26.314


In [348]:
comparisons[comparisons.Contrast.str.contains("Long deceleration")]

,Contrast,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
3,Constant speed - Long deceleration,Stay,-0.160,-0.309,-0.011,0.055,2978.218,-2.933,0.028,*
6,Long acceleration - Long deceleration,Stay,-0.235,-0.383,-0.086,0.054,2980.403,-4.314,0.000,***
8,Acceleration nudge - Long deceleration,Stay,-0.285,-0.435,-0.135,0.055,2980.974,-5.184,0.000,***
9,Deceleration nudge - Long deceleration,Stay,-0.098,-0.249,0.054,0.055,2974.938,-1.758,0.399,
13,Constant speed - Long deceleration,Go,-0.259,-0.396,-0.122,0.050,2975.828,-5.154,0.000,***
16,Long acceleration - Long deceleration,Go,-0.303,-0.459,-0.146,0.057,2977.957,-5.275,0.000,***
18,Acceleration nudge - Long deceleration,Go,-0.224,-0.366,-0.083,0.052,2975.561,-4.322,0.000,***
19,Deceleration nudge - Long deceleration,Go,-0.263,-0.393,-0.133,0.047,2974.297,-5.539,0.000,***


In [347]:
comparisons[comparisons.Contrast.str.contains("Acceleration nudge")]

,Contrast,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
1,Constant speed - Acceleration nudge,Stay,0.125,0.007,0.243,0.043,2974.144,2.884,0.032,*
4,Long acceleration - Acceleration nudge,Stay,0.051,-0.066,0.167,0.043,2973.373,1.180,0.763,
7,Acceleration nudge - Deceleration nudge,Stay,-0.188,-0.311,-0.064,0.045,2978.270,-4.149,0.000,***
8,Acceleration nudge - Long deceleration,Stay,-0.285,-0.435,-0.135,0.055,2980.974,-5.184,0.000,***
11,Constant speed - Acceleration nudge,Go,-0.035,-0.188,0.118,0.056,2973.159,-0.618,0.972,
14,Long acceleration - Acceleration nudge,Go,-0.079,-0.249,0.092,0.062,2974.195,-1.258,0.717,
17,Acceleration nudge - Deceleration nudge,Go,0.039,-0.108,0.186,0.054,2973.776,0.720,0.952,
18,Acceleration nudge - Long deceleration,Go,-0.224,-0.366,-0.083,0.052,2975.561,-4.322,0.000,***


In [346]:
comparisons[comparisons.Contrast.str.contains("Deceleration nudge")]

,Contrast,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
2,Constant speed - Deceleration nudge,Stay,-0.063,-0.185,0.060,0.045,2975.095,-1.396,0.630,
5,Long acceleration - Deceleration nudge,Stay,-0.137,-0.258,-0.016,0.044,2977.830,-3.081,0.018,*
7,Acceleration nudge - Deceleration nudge,Stay,-0.188,-0.311,-0.064,0.045,2978.270,-4.149,0.000,***
9,Deceleration nudge - Long deceleration,Stay,-0.098,-0.249,0.054,0.055,2974.938,-1.758,0.399,
12,Constant speed - Deceleration nudge,Go,0.004,-0.139,0.147,0.052,2974.058,0.080,1.000,
15,Long acceleration - Deceleration nudge,Go,-0.040,-0.201,0.122,0.059,2975.242,-0.672,0.963,
17,Acceleration nudge - Deceleration nudge,Go,0.039,-0.108,0.186,0.054,2973.776,0.720,0.952,
19,Deceleration nudge - Long deceleration,Go,-0.263,-0.393,-0.133,0.047,2974.297,-5.539,0.000,***


# Negative rating

In [372]:
model_rating = pymer4.models.Lmer("is_negative_rating ~ tta_0 + decision + condition +  (1 | subj_id)", data=exp_measures, family="binomial")
# model_rating = pymer4.models.Lmer("is_negative_rating ~ decision + condition + (1 | subj_id) ", data=exp_measures, family="binomial")
model_rating_fit = model_rating.fit(factors={"condition": ["Constant speed", "Long acceleration", "Acceleration nudge", "Deceleration nudge", "Long deceleration"], "decision": ["Stay", "Go"]}, summarize=True)

D:\source\cognitive-av\venv\lib\site-packages\rpy2\robjects\pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: is_negative_rating~tta_0+decision+condition+(1|subj_id)

Family: binomial	 Inference: parametric

Number of observations: 3800	 Groups: {'subj_id': 19.0}

Log-likelihood: -590.305 	 AIC: 1196.610

Random effects:

                Name    Var    Std
subj_id  (Intercept)  1.302  1.141

No random effect correlations specified

Fixed effects:



In [373]:
df = model_rating.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]

p_func = lambda p: "{:.2f}".format(p) if p>0.001 else "$<0.001$"

df["P-val"] = df["P-val"].apply(p_func)

styler = df.rename(columns=column_names, index=var_names).style.format(precision=2)

with open('figures/tab_rating.tex', 'w') as tf:
     tf.write(styler.to_latex(
         column_format="rrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:rating", caption="Coefficients of the mixed-effects logistic regression describing occurrence of the negative rating as a function of $\\textrm{TTA}_0$ and condition (with Constant speed as the reference category). Participant ID was included as a random intercept."
     )
)

In [376]:
marginal_estimates, comparisons = model_rating.post_hoc(marginal_vars=["condition"])
# "Cell" means of the ANOVA
marginal_estimates

P-values adjusted by tukey method for family of 10 estimates


,condition,Estimate,2.5_ci,97.5_ci,SE,DF
0,Constant speed,-5.101,-5.879,-4.323,0.397,inf
1,Long acceleration,-5.161,-5.929,-4.393,0.392,inf
2,Acceleration nudge,-4.583,-5.293,-3.873,0.362,inf
3,Deceleration nudge,-4.310,-5.004,-3.617,0.354,inf
4,Long deceleration,-1.730,-2.303,-1.157,0.292,inf


In [363]:
comparisons

,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,Z-stat,P-val,Sig
0,Constant speed - Long acceleration,0.060,-0.946,1.066,0.369,inf,0.163,1.000,
1,Constant speed - Acceleration nudge,-0.517,-1.446,0.411,0.340,inf,-1.520,0.549,
2,Constant speed - Deceleration nudge,-0.790,-1.694,0.113,0.331,inf,-2.387,0.119,
3,Constant speed - Long deceleration,-3.371,-4.194,-2.547,0.302,inf,-11.163,0.000,***
4,Long acceleration - Acceleration nudge,-0.578,-1.487,0.332,0.333,inf,-1.732,0.414,
5,Long acceleration - Deceleration nudge,-0.851,-1.735,0.034,0.324,inf,-2.623,0.066,.
6,Long acceleration - Long deceleration,-3.431,-4.238,-2.624,0.296,inf,-11.599,0.000,***
7,Acceleration nudge - Deceleration nudge,-0.273,-1.068,0.522,0.292,inf,-0.937,0.883,
8,Acceleration nudge - Long deceleration,-2.853,-3.551,-2.155,0.256,inf,-11.150,0.000,***
9,Deceleration nudge - Long deceleration,-2.580,-3.242,-1.919,0.242,inf,-10.641,0.000,***
